In [8]:
# The packages
import pandas as pd
import sqlite3

### Une inspection des données des incendies

In [2]:
# 1. Chargement de CSV brut des incendies
df_incendies = pd.read_csv("../data/csvs/incendies/incendie_1.csv", sep=";")

In [15]:
df_incendies.head()

,Année,Numéro,Département,Code INSEE,Nom de la commune,Date de première alerte,Surface parcourue (m2),Surface forêt (m2),Surface maquis garrigues (m2),Autres surfaces naturelles hors forêt (m2),...,Surfaces non boisées artificialisées (m2),Surfaces non boisées (m2),Précision des surfaces,Type de peuplement,Nature,Décès ou bâtiments touchés,Nombre de décès,Nombre de bâtiments totalement détruits,Nombre de bâtiments partiellement détruits,Précision de la donnée
0,1978,636,11,11262,Narbonne,1978-01-01 03:20:00,1000,0,0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1978,750,13,13051,Lançon-Provence,1978-01-01 12:51:00,8000,0,0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1978,3779,83,83063,La Garde-Freinet,1978-01-01 16:10:00,15000,0,0,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1978,3780,83,83126,La Seyne-sur-Mer,1978-01-03 16:15:00,1000,0,0,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1978,751,13,13085,Roquefort-la-Bédoule,1978-01-03 17:31:00,1000,0,0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Les dimensions du DataFrame
df_incendies.shape

(18473, 24)

### Une inspection des données des communes

In [4]:
# 1. Chargement de CSV brut des incendies
df_communes = pd.read_csv("../data/csvs/communes/communes.csv")
df_communes.head()

/tmp/ipykernel_29953/1689902706.py:2: DtypeWarning: Columns (1,12,14,16,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_communes = pd.read_csv("../data/csvs/communes/communes.csv")


,Unnamed: 0,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,...,longitude_mairie,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,niveau_equipements_services,niveau_equipements_services_texte,gentile,url_wikipedia,url_villedereve
0,0,01001,L'Abergement-Clémenciat,Abergement-Clémenciat,à Abergement-Clémenciat,de l'Abergement-Clémenciat,l-abergement-clemenciat,L'ABERGEMENT-CLÉMENCIAT,COM,commune,...,4.921,46.153,4.926,6,Rural à habitat dispersé,0.0,communes non pôle,NaN,https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01001-l-abergemen...
1,1,01002,L'Abergement-de-Varey,Abergement-de-Varey,à Abergement-de-Varey,de l'Abergement-de-Varey,l-abergement-de-varey,L'ABERGEMENT-DE-VAREY,COM,commune,...,5.423,46.009,5.428,6,Rural à habitat dispersé,0.0,communes non pôle,"Abergementais, Abergementaises",https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01002-l-abergemen...
2,2,01004,Ambérieu-en-Bugey,Ambérieu-en-Bugey,à Ambérieu-en-Bugey,d'Ambérieu-en-Bugey,amberieu-en-bugey,AMBÉRIEU-EN-BUGEY,COM,commune,...,5.360,45.961,5.373,2,Centres urbains intermédiaires,3.0,centres structurants d'équipements et de services,"Ambarrois, Ambarroises",https://fr.wikipedia.org/wiki/fr:Ambérieu-en-B...,https://villedereve.fr/ville/01004-amberieu-en...
3,3,01005,Ambérieux-en-Dombes,Ambérieux-en-Dombes,à Ambérieux-en-Dombes,d'Ambérieux-en-Dombes,amberieux-en-dombes,AMBÉRIEUX-EN-DOMBES,COM,commune,...,4.903,45.996,4.912,5,Bourgs ruraux,1.0,centres locaux d'équipements et de services,Ambarrois,https://fr.wikipedia.org/wiki/fr:Ambérieux-en-...,https://villedereve.fr/ville/01005-amberieux-e...
4,4,01006,Ambléon,Ambléon,à Ambléon,d'Ambléon,ambleon,AMBLÉON,COM,commune,...,5.601,45.750,5.594,6,Rural à habitat dispersé,0.0,communes non pôle,Ambléonais,https://fr.wikipedia.org/wiki/fr:Ambléon,https://villedereve.fr/ville/01006-ambleon


In [5]:
df_communes.shape

(34935, 47)

In [25]:
# Détection automatique des colonnes avec plusieurs types
colonnes_probleme = [
    col for col in df_communes.columns 
    if df_communes[col].map(type).nunique() > 1
]

print("Colonnes avec des types mélangés :", colonnes_probleme)

# Afficher seulement ces colonnes
df_communes[colonnes_probleme].head()

Colonnes avec des types mélangés : ['code_insee', 'dep_code', 'canton_code', 'canton_nom', 'epci_code', 'epci_nom', 'codes_postaux', 'code_insee_centre_zone_emploi', 'code_unite_urbaine', 'nom_unite_urbaine', 'type_commune_unite_urbaine', 'statut_commune_unite_urbaine', 'niveau_equipements_services_texte', 'gentile', 'url_wikipedia']


,code_insee,dep_code,canton_code,canton_nom,epci_code,epci_nom,codes_postaux,code_insee_centre_zone_emploi,code_unite_urbaine,nom_unite_urbaine,type_commune_unite_urbaine,statut_commune_unite_urbaine,niveau_equipements_services_texte,gentile,url_wikipedia
0,01001,01,0108,Châtillon-sur-Chalaronne,200069193,CC de la Dombes,01400,01053,01000,NaN,HORS UNITE URBAINE,H,communes non pôle,NaN,https://fr.wikipedia.org/wiki/fr:L'Abergement-...
1,01002,01,0101,Ambérieu-en-Bugey,240100883,CC de la Plaine de l'Ain,01640,01053,01000,NaN,HORS UNITE URBAINE,H,communes non pôle,"Abergementais, Abergementaises",https://fr.wikipedia.org/wiki/fr:L'Abergement-...
2,01004,01,0101,Ambérieu-en-Bugey,240100883,CC de la Plaine de l'Ain,"01500, 01501, 01504, 01503, 01502, 01505, 01506",01053,01303,Ambérieu-en-Bugey,UNITE URBAINE,C,centres structurants d'équipements et de services,"Ambarrois, Ambarroises",https://fr.wikipedia.org/wiki/fr:Ambérieu-en-B...
3,01005,01,0122,Villars-les-Dombes,200042497,CC Dombes Saône Vallée,01330,69264,01000,NaN,HORS UNITE URBAINE,H,centres locaux d'équipements et de services,Ambarrois,https://fr.wikipedia.org/wiki/fr:Ambérieux-en-...
4,01006,01,0104,Belley,200040350,CC Bugey Sud,01300,01034,01000,NaN,HORS UNITE URBAINE,H,communes non pôle,Ambléonais,https://fr.wikipedia.org/wiki/fr:Ambléon


In [26]:
# Détection des colonnes problématiques
colonnes_probleme = [
    col for col in df_communes.columns 
    if df_communes[col].map(type).nunique() > 1
]

# Afficher les valeurs uniques pour chaque colonne concernée
for col in colonnes_probleme:
    print(f"\n--- Colonne : {col} ---")
    print(df_communes[col].unique())



--- Colonne : code_insee ---
['01001' '01002' '01004' ... 97615 97616 97617]

--- Colonne : dep_code ---
['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '2A' '2B' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41'
 '42' '43' '44' 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62
 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86
 87 88 89 90 91 92 93 94 95 971 972 973 974 976]

--- Colonne : canton_code ---
['0108' '0101' '0122' ... 97699.0 97613.0 97611.0]

--- Colonne : canton_nom ---
['Châtillon-sur-Chalaronne' 'Ambérieu-en-Bugey' 'Villars-les-Dombes' ...
 'Mamoudzou' 'Tsingoni' 'Pamandzi']

--- Colonne : epci_code ---
['200069193' '240100883' '200042497' ... 200059871 200060457 200050532]

--- Colonne : epci_nom ---
['CC de la Dombes' "CC de la Plaine de l'Ain" 'CC Dombes Saône Vallée' ...
 'CC du Centre-Ouest' 'CA de Dembeni / Mamoudzou' 'CC de Peti

In [13]:
# chemin vers ta base SQLite
db_path = "../data/csvs/incendies.db"

# connexion
con = sqlite3.connect(db_path)

# lire une table
df = pd.read_sql("SELECT * FROM incendies_stage LIMIT 10;", con)

# afficher les 5 premières lignes
df.head()


,id,date,annee,mois,jour,heure,code_insee,commune,departement,region,cause,type_vegetation,surface_brulee,code_insee_norm
0,Année,Numéro,Département,Code INSEE,Nom de la commune,Date de première alerte,Surface parcourue (m2),Surface forêt (m2),Surface maquis garrigues (m2),Autres surfaces naturelles hors forêt (m2),Surfaces agricoles (m2),Autres surfaces (m2),Surface autres terres boisées (m2),Surfaces non boisées naturelles (m2)
1,1978,636,11,11262,Narbonne,1978-01-01 03:20:00,1000,0,0,,,,,
2,1978,750,13,13051,Lançon-Provence,1978-01-01 12:51:00,8000,0,0,,,,,
3,1978,3779,83,83063,La Garde-Freinet,1978-01-01 16:10:00,15000,0,0,,,,,
4,1978,3780,83,83126,La Seyne-sur-Mer,1978-01-03 16:15:00,1000,0,0,,,,,


In [ ]:
# n’oublie pas de fermer
con.close()